## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-06-13-03-08 +0000,bbc,Bank says inflation has 'peaked' as it holds i...,https://www.bbc.com/news/articles/c77zkg5504eo...
1,2025-11-06-12-59-00 +0000,wsj,UPS Plane That Crashed Was 34 Years Old—and Re...,https://www.wsj.com/business/airlines/ups-plan...
2,2025-11-06-12-58-55 +0000,nyt,The U.S. Is Skipping This Year’s Climate Summi...,https://www.nytimes.com/2025/11/06/climate/bel...
3,2025-11-06-12-50-35 +0000,bbc,World’s oldest president sworn in for eighth t...,https://www.bbc.com/news/articles/cj978zlgp7go...
4,2025-11-06-12-50-00 +0000,wsj,"Nissan Remains in Red Amid Restructuring, Tari...",https://www.wsj.com/business/autos/nissan-moto...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2306/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
333,trump,53
189,mamdani,34
124,new,27
207,election,19
100,shutdown,18


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
129,2025-11-06-05-58-57 +0000,nypost,Florida sheriff invites New Yorkers looking to...,https://nypost.com/2025/11/06/us-news/florida-...,166
197,2025-11-05-23-44-26 +0000,nyt,Trump vs. Mamdani: President Sees a Capable Fo...,https://www.nytimes.com/2025/11/05/us/politics...,150
276,2025-11-05-20-26-30 +0000,nypost,New York Post’s cover on Zohran Mamdani electi...,https://nypost.com/2025/11/05/us-news/new-york...,141
282,2025-11-05-20-10-00 +0000,wsj,Zohran Mamdani sealed his mayoral victory in N...,https://www.wsj.com/politics/elections/the-vot...,136
167,2025-11-06-02-00-00 +0000,wsj,Economic discontent helped deliver election wi...,https://www.wsj.com/politics/elections/economi...,134


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
129,166,2025-11-06-05-58-57 +0000,nypost,Florida sheriff invites New Yorkers looking to...,https://nypost.com/2025/11/06/us-news/florida-...
376,128,2025-11-05-14-42-00 +0000,wsj,Lawmakers are hopeful they can reach a biparti...,https://www.wsj.com/politics/policy/lawmakers-...
234,65,2025-11-05-22-07-00 +0000,wsj,"Maine Rep. Jared Golden, one of the most vulne...",https://www.wsj.com/politics/elections/jared-g...
372,64,2025-11-05-15-00-06 +0000,nyt,A Supreme Court Veteran and a Newcomer Will Ma...,https://www.nytimes.com/2025/11/05/us/politics...
161,53,2025-11-06-02-22-10 +0000,nypost,Erika Kirk recalls harrowing moment she found ...,https://nypost.com/2025/11/05/us-news/erika-ki...
258,48,2025-11-05-21-21-00 +0000,wsj,Trump urged Republican senators to jettison th...,https://www.wsj.com/politics/policy/trumps-ans...
216,44,2025-11-05-22-53-33 +0000,nypost,‘Tutors’ at Musk startup xAI had to give up ri...,https://nypost.com/2025/11/05/business/tutors-...
42,43,2025-11-06-11-17-29 +0000,cbc,Vietnam evacuates hundreds of thousands ahead ...,https://www.cbc.ca/news/world/typhoon-kalmaegi...
65,43,2025-11-06-10-46-38 +0000,nypost,FAA reveals airports facing flight cancellatio...,https://nypost.com/2025/11/06/us-news/faa-reve...
240,42,2025-11-05-21-57-22 +0000,wapo,Black boxes recovered in UPS plane crash in Lo...,https://www.washingtonpost.com/nation/2025/11/...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
